In [1]:
import numpy as np
import pandas as pd

In [14]:
# From stock_price_adj to fill in missing dates
df1 = pd.read_csv("XXX_adj.csv")

# Manual scraping of revenu, net income and shares outstanding for each company's quarterly report
df2 = pd.read_excel("XXX_financials.xlsx")

In [15]:
# Cleanup
df1.columns = df1.columns.str.replace(' ', '')
df1.rename(columns = {'AdjClose':'Price'}, inplace = True)
df1["Date"] = pd.to_datetime(df1["Date"])

# Merge
df3 = pd.merge(df1,df2,
               how='left',left_on='Date',right_on='Date', validate = 'm:m')

In [17]:
# EPS calculation
df3['EPS'] = df3['Net_income'] / df3['Shares_outstanding']

In [19]:
# Heikin_ashi calculations

shift_open = df3.Open.shift(1)
shift_close = df3.Price.shift(1)

# The midpoint of the previous day price
heikin_open = (shift_open + shift_close)/2

# The average of the current day price
heikin_close = df3.apply(lambda column: (column.Open + column.Price
                       + column.High + column.Low)/4 , axis=1)

# Heikin_close price greater than heikin_open price means trending up
df3['Heikin_trend'] = np.where(heikin_close>heikin_open, 'uptrend', 'downtrend')

In [21]:
#Other cleanup

# Change volume to millions
df3['Volume'] = df3['Volume']/1000000

# PE_Ratio
df3['PE_ratio'] = df3['Price'] / df3['EPS']

df3 = df3.drop(['Open', 'High','Low', 'Close',
               'Net_income','Shares_outstanding'], axis=1)

In [24]:
# Exporting CSV after feature engineering of financial data

df3.to_csv('XXX.csv', index=False)

In [49]:
# Price not in millions

# Revenue may be a good indicator of 
#consumer demand of the product (could be sentiment-case too)